In [46]:
import pandas as pd
import seaborn
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import spacy
import string
from spacy.lang.en import English
from nltk.stem.porter import PorterStemmer
spacy.load('en')
parser = English()

In [32]:
STOPWORDS = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS)) 
SYMBOLCHARS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","''"]

In [33]:
hateSpeechDF = pd.read_csv('../Data/Hate Speech and Offensive Language Dataset/Cleaned_labeled_data.csv')
spamSmsDF = pd.read_csv('../Data/SMS Spam Dataset/Cleaned_SMSSpamCollection.csv')

In [34]:
hateSpeechDF.head()

,class,category,text
0,2,Neither,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,OffensiveLang,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,OffensiveLang,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,OffensiveLang,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,OffensiveLang,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [35]:
spamSmsDF.head()

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [47]:
def tokenizeText(textData):

    textData = textData.strip().replace("\n", " ").replace("\r", " ")
    textData = textData.lower()
    tokens = parser(textData)

    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # Remove Stop Words
    tokens = [tok for tok in tokens if tok.lower() not in STOPWORDS]
    # Remove Symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLCHARS]
    # Remove words with less than 3 characters
    tokens = [tok for tok in tokens if len(tok) >= 3]
    # Remove Non-Alphabetic Characters
    tokens = [tok for tok in tokens if tok.isalpha()]
    
    # Stemming of Words
    porter = PorterStemmer()
    tokens = [porter.stem(word) for word in tokens]
    
    tokens = list(set(tokens))
    textData = ' '.join(tokens[:])
    return textData

In [48]:
hateSpeechDF['text'] = hateSpeechDF['text'].apply(lambda x:tokenizeText(x))
hateSpeechDF['text']

0                  clean woman trash man complain amp hous
1               dat tyga cuffin cold dwn hoe bad boy place
2                        dawg start bitch fuck confus shit
3                                         like tranni look
4                          true bitch faker shit hear told
                               ...                        
24778                        right trash bibl lie scriptur
24779      babi crazi gone broke drove redneck wrong heart
24780       like nigguh fuckin dat wanna young buck eat di
24781                       tellin bitch wild lie got youu
24782    color combin white orang yellow dahlia coll ei...
Name: text, Length: 24783, dtype: object

In [49]:
spamSmsDF['text'] = spamSmsDF['text'].apply(lambda x:tokenizeText(x))
spamSmsDF['text']

0       great point cine world crazi buffet got wat av...
1                                        lar oni wif joke
2       final tkt txt free comp win entri text cup rec...
3                                       hor dun earli say
4                                  goe nah live usf think
                              ...                        
5567    rate nation tri time easi minut contact pound ...
5568                                     esplanad go home
5569                                    piti mood suggest
5570       like bitch free gave buy act guy week interest
5571                                            true rofl
Name: text, Length: 5572, dtype: object